## Connect to IB Gateway

`util.startLoop()` is required to run the event loop in a separate thread. This is needed for the `ib.sleep()` function to work.
TODO: change localhost to server IP

In [3]:
import time

from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('localhost', 4002, clientId=1)

KeyboardInterrupt: 

## Check IB Gateway connection|

In [ ]:
ib.isConnected()

## Check ability to disconnect

In [ ]:
ib.disconnect()
ib.isConnected()

# Reconnect

In [5]:
ib.connect('localhost', 7497, clientId=101)

<IB connected to localhost:7497 clientId=101>

# Get account summary

In [6]:
accountSummary = ib.accountSummary()
accountSummary

[AccountValue(account='DU6339390', tag='AccountType', value='INDIVIDUAL', currency='', modelCode=''),
 AccountValue(account='DU6339390', tag='Cushion', value='1', currency='', modelCode=''),
 AccountValue(account='DU6339390', tag='DayTradesRemaining', value='-1', currency='', modelCode=''),
 AccountValue(account='DU6339390', tag='DayTradesRemainingT+1', value='-1', currency='', modelCode=''),
 AccountValue(account='DU6339390', tag='DayTradesRemainingT+2', value='-1', currency='', modelCode=''),
 AccountValue(account='DU6339390', tag='DayTradesRemainingT+3', value='-1', currency='', modelCode=''),
 AccountValue(account='DU6339390', tag='DayTradesRemainingT+4', value='-1', currency='', modelCode=''),
 AccountValue(account='DU6339390', tag='LookAheadNextChange', value='0', currency='', modelCode=''),
 AccountValue(account='DU6339390', tag='AccruedCash', value='3448.74', currency='USD', modelCode=''),
 AccountValue(account='DU6339390', tag='AvailableFunds', value='1012580.10', currency='US

In [14]:
# verify we are trading a paper account
accountSummary[0].account
# 'DU6339390' is my paper trading account with IB

'DU6339390'

# Lets Request some historical data

In [ ]:
import numpy as np
bars_spy = ib.reqHistoricalData(
            contract=Stock('SPY', 'SMART', 'USD'),
            endDateTime='',
            durationStr='60 s',
            barSizeSetting='1 secs',
            whatToShow='MIDPOINT',
            useRTH=True,
            formatDate=1)
df_spy = util.df(bars_spy)
avg_price = round(np.nanmean(df_spy['close']), 2)

In [ ]:
# helper function to round to nearest multiple
def round_to_multiple(number, multiple, direction="up"):
    if direction == "up":
        return int(np.ceil(number / multiple)) * multiple
    elif direction == "down":
        return int(np.floor(number / multiple)) * multiple
    else:
        raise ValueError("direction must be either up or down")

In [ ]:
# helper function to get the nearest 45 DTE expiration
import datetime
def getDaysToNearest45DTE():
    spy = Stock('SPY', 'SMART', 'USD')
    ib.qualifyContracts(spy)
    ib.reqMarketDataType(4)

    chains = ib.reqSecDefOptParams(spy.symbol, '', spy.secType, spy.conId)
    chain = next(c for c in chains if c.exchange == 'SMART')

    # get the nearest monthly expiration
    today = datetime.date.today()
    targetDTE = today + datetime.timedelta(days=45)

    # convert chain.expirations to datetime.date
    expire = [datetime.datetime.strptime(exp, '%Y%m%d').date() for exp in chain.expirations]

    # find the nearest monthly expiration in chain.expirations to targetDTE
    nearestDTE = min(expire, key=lambda x: abs(x - targetDTE))

    # find the number of days until the nearest monthly expiration
    daysToexp = (nearestDTE - today).days/365

    return daysToexp, nearestDTE

In [ ]:
# helper function to get spot price of underlying
def getSpotPrice(stockName):
    # get the current price of ticker until we have access to historical data
    stock = Stock(stockName, 'SMART', 'USD')
    ib.qualifyContracts(stock)

    ib.reqMarketDataType(4)
    [ticker] = ib.reqTickers(stock)
    return ticker.marketPrice()

In [ ]:
# helper function to get 16 delta call
def getCalltoSell(delta):
    spotSPY = getSpotPrice('SPY')
    currentIV = getSpotPrice('VIX')/100  # or spy_iv from py_vollib or IV from ib_insync
    # spy_iv = implied_volatility(ATMOptionPrice, spy_price, ATMStrike, DTE/365, ir=0.00, 0.00, 'c')
    # print("Implied volatility of SPY: ", spy_iv)
    # spy_iv = ib.calculateImpliedVolatility(Option('SPY', '20201218', 350, 'C', 'SMART'), 350)
    annualTime, expiration = getDaysToNearest45DTE()
    ir = 0.00

    for i in range(0, 1000):
        df = pd.DataFrame()
        df['Flag'] = ['c']        # 'c' for call, 'p' for put
        df['S'] = spotSPY         # Underlying asset price
        df['K'] = [spotSPY + i]   # Strike(s)
        df['T'] = annualTime            # (Annualized) time-to-expiration
        df['R'] = ir              # Interest free rate
        df['IV'] = currentIV      # Implied Volatility
        result = price_dataframe(df, flag_col='Flag', underlying_price_col='S', strike_col='K', annualized_tte_col='T',
                             riskfree_rate_col='R', sigma_col='IV', model='black_scholes', inplace=False)
        if result['delta'][0] <= delta:
            # print(result['delta'])
            # print(round_to_multiple(df['K'][0],5,"up"))
            callToSell = round_to_multiple(df['K'][0],5,"up")
            # print(df['K'][0])
            break

    return callToSell

In [ ]:
# helper function to get 16 delta put
import pandas as pd
import numpy as np
from py_vollib_vectorized import price_dataframe

def getPuttoSell(delta):
    spotSPY = getSpotPrice('SPY')
    currentIV = getSpotPrice('VIX')/100  # or spy_iv from py_vollib
    # calculate the implied volatility of the current price of SPY
    # spy_iv = implied_volatility(ATMOptionPrice, spy_price, ATMStrike, DTE/365, ir=0.00, 0.00, 'c')
    # print("Implied volatility of SPY: ", spy_iv)
    annualTime, expiration = getDaysToNearest45DTE()
    ir = 0.00

    for i in range(0, 1000):
        df = pd.DataFrame()
        df['Flag'] = ['p']        # 'c' for call, 'p' for put
        df['S'] = spotSPY         # Underlying asset price
        df['K'] = [spotSPY - i]   # Strike(s)
        df['T'] = annualTime            # (Annualized) time-to-expiration
        df['R'] = ir              # Interest free rate
        df['IV'] = currentIV      # Implied Volatility
        result = price_dataframe(df, flag_col='Flag', underlying_price_col='S', strike_col='K', annualized_tte_col='T',
                             riskfree_rate_col='R', sigma_col='IV', model='black_scholes', inplace=False)
        if result['delta'][0] >= -1*delta:
            # print(result['delta'])
            # print(round_to_multiple(df['K'][0],5,"down"))
            putToSell = round_to_multiple(df['K'][0],5,"down")
            # print(df['K'][0])
            break

    return putToSell

# Use Py_vollib to calculate the implied volatility of the current price of SPY
### We will use this to calculate the strike prices for our strangle

In [ ]:
from py_vollib.black_scholes_merton.implied_volatility import implied_volatility
from py_vollib.black_scholes_merton.greeks.analytical import delta
from py_vollib.black_scholes_merton import black_scholes_merton as bsm
from scipy.stats import norm
import math

# Pseudo code for now until we have the data

# calculate the implied volatility of the current price of SPY
spy_iv = implied_volatility(ATMOptionPrice, spy_price, ATMStrike, DTE/365, ir=0.00, 0.00, 'c')
print("Implied volatility of SPY: ", spy_iv)

# Demostating in pseudo code how to create a bag of contracts so it all executes at once

In [ ]:
contract = Contract()
contract.symbol = spy_call.symbol
contract.secType = 'BAG'
contract.currency = spy_call.currency
contract.exchange = spy_call.exchange

leg1 = ComboLeg()
leg1.conId = spy_call.conId
leg1.ratio = 1
leg1.action = 'SELL'
leg1.exchange = spy_call.exchange

leg2 = ComboLeg()
leg2.conId = spy_put.conId
leg2.ratio = 1
leg2.action = 'SELL'
leg2.exchange = spy_put.exchange

contract.comboLegs = [leg1, leg2]

# Get the price of the bag and make it a market order

In [ ]:
bars = ib.reqHistoricalData(
            contract=contract,
            endDateTime='',
            durationStr='60 s',
            barSizeSetting='1 secs',
            whatToShow='TRADES',
            useRTH=True,
            formatDate=1)
df = util.df(bars)
avg_price = round(np.nanmean(df['close']), 2)

In [ ]:
order = LimitOrder('BUY', 1, round(avg_price*0.995,2), tif='DAY', account = accountSummary[0].account)
trade = ib.placeOrder(contract, order)
print(trade)
time.sleep(5)
ib.disconnect()